In [1]:
%pwd

'c:\\Users\\Hp\\Documents\\Wine_Quality_End_To_End\\research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\Hp\\Documents\\Wine_Quality_End_To_End'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [6]:
# import yaml

In [9]:
from src.WineQuality.constant import *
from src.WineQuality.utils.common import read_yaml, create_directories


ModuleNotFoundError: No module named 'yaml'

In [7]:
# CONFIG_FILE_PATH
# read_yaml(CONFIG_FILE_PATH)
# #config = read_yaml(CONFIG_FILE_PATH)
# #config.artifacts_root

In [8]:
class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH
                ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
    
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from WineQuality import logger
from WineQuality.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f'{filename} downloaded with following info: \n{headers}')
        else:
            logger.info(f'File alraedy exists of size: {get_size(Path(self.config.local_data_file))}')
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2023-11-13 22:59:11,446:  INFO:  common : yaml file : config\config.yaml loaded successfully]
[2023-11-13 22:59:11,446:  INFO:  common : yaml file : params.yaml loaded successfully]
[2023-11-13 22:59:11,454:  INFO:  common : yaml file : schema.yaml loaded successfully]
[2023-11-13 22:59:11,454:  INFO:  common : Created directories at: artifacts]
[2023-11-13 22:59:11,462:  INFO:  common : Created directories at: artifacts/data_ingestion]
[2023-11-13 22:59:11,462:  INFO:  1607146777 : File alraedy exists of size: ~22 KB]
